In [ ]:
# Uncomment below section and run in case of re-connecting Colab

# !pip install git+https://github.com/haven-jeon/PyKoSpacing.git
# !pip install transformers
# !pip install git+https://github.com/ssut/py-hanspell.git

# from google.colab import drive
# drive.mount('/content/drive')

# %cd drive/MyDrive/MBTI
# !pwd

In [6]:
import os
import re

import torch
import pandas as pd
from torch.utils.data import Dataset
from pykospacing import Spacing                     #FIXME: 이 패키지가 m1 mac 에서 작동을 안 함... -> Colab 에서 불러서 실행할 것
from hanspell import spell_checker
from transformers import AutoTokenizer

pd.set_option('display.width', 180)

#TODO:
# * train / text 다른 로직이 필요 (데이터 형식이 조금 다름)
# * 다른 Text preprocess 방식 도입 검토

class ddDataset(Dataset):
    def __init__(
        self,
        data_path: str,
        question_path: str,
        pretrained_url: str = "klue/bert-base",
        target_mbti: str = None,
        txt_preprocess: bool = True,
        normalize: bool = True,
        is_binary_classification: bool = True,
        is_bert: bool = True,
        is_train: bool = True
        ):
        """DataLoader for MBTI dataset

        Args:
            data_path (str): Data file path. Both csv and parguet files are allowed.
            question_path (str): Question file path. Both csv and parguet files are allowed.
            target_mbti (str): Target mbti for binary classification.
            txt_preprocess (bool, optional): Text preprocessing pipeline. (e.g. fixing grammar, removing punctuations). Defaults to True.
            normalize (bool, optional): Normalize numeric attribute. Defaults to True.
            is_binary_classification (bool, optional): Target of task. You can choose btw Multi-class classificaiton
                and 4 binary classification problem. Defaults to True.
            is_bert (bool, optional): Using BERT for language model or not. Defaults to True.
            is_train (bool, optional): Whether given data is for training or not. Defaults to True.
        """

        data = pd.read_csv(data_path) if data_path.endswith('.csv') else pd.read_parquet(data_path)
        self.question_data = pd.read_csv(question_path)

        print("data :", data.head())
        print("--------origin---------")
        data_origin = data.copy()

        # preprocess data
        if txt_preprocess:
            data['Answer'] = data['Answer'].apply(self.fix_grammar)
            data['Answer'] = data['Answer'].apply(self.fix_spacing)
            print(data['Answer'].compare(data_origin['Answer']))
            data['Answer'] = data['Answer'].apply(self.remove_punctuation)
        if normalize:
            data['Age'] = (data['Age'] - data['Age'].mean()) / data['Age'].std()


        print("--------preprocess---------")

        # align dataset with binary classification (only for training data - test data doesn't contain 'MBTI' field)
        label_col = None
        if is_train and is_binary_classification:
            label_col = self.prepare_binary_classification(data, target_mbti)
            l_list = label_col.split('/')
            assert data[label_col].value_counts()[0] == \
                  data[label_col].value_counts()[1]

        print("data :", data.head())
        print("--------prepare_binary_classification---------")

        # prepare for language model
        #TODO: tokenizer class 인자로 넣어야 함 & tokenizer 만으로 [CLS], [SEP] 잘 붙는지 확인 필요
        self.tokenizer = AutoTokenizer.from_pretrained(pretrained_url)
        data = self.tokenize(data)
        

        selected_question = self.question_data.iloc[data_origin['Q_number'][0] - 1].Question
        selected_answer = data_origin['Answer'][0]
        print(selected_question)
        print(selected_answer)
        encoding = self.tokenizer(selected_question, selected_answer)
        print('input_ids :', encoding['input_ids'])
        print('token_type_ids :', encoding['token_type_ids'])
        print('attention_mask :', encoding['attention_mask'])
        decoding = self.tokenizer.convert_ids_to_tokens(encoding['input_ids'])
        print(decoding)


        print("data :", data.head())
        print("--------tokenize---------")

        return

        # select columns for both training and inference
        #TODO: 테스트 데이터를 고려해서 유저 정보를 학습에 활용하지 않는 상황. 필요시 고쳐야 함.
        self.selected_cols = ['Gender', 'Age', 'QandA']
        self.label_col = label_col
        self.data = data

    def __len__(self):
        return len(self.data)

    def __getitem__(self, idx):
        #TODO:
        # * getitem --> BERT input dimension 확인 후 변환 & 다른 features 들과 붙여서 input instance 생성
        
        pass

    # ======================
    #    Helper Functions
    # ======================

    def fix_grammar(self, answer: str) -> str:
        answer = spell_checker.check(answer)
        return answer.checked

    def fix_spacing(self, answer: str) -> str:
        answer = answer.replace(" ", '')
        spacing = Spacing()
        return spacing(answer)

    def remove_punctuation(self, answer: str) -> str:
        #FIXME: remove punctuation 검증 필요
        answer = re.sub(r'[@%\\*=()/~#&\+á?\xc3\xa1\-\|\.\:\;\!\-\,\_\~\$\'\"]', '', answer)
        answer = re.sub(r"^\s+", '', answer)                    # remove space from start
        answer = re.sub(r'\s+$', '', answer)                    # remove space from the end
        return answer

    def prepare_binary_classification(self, data: pd.DataFrame, target_mbti: str) -> str:
        t_value, f_value = 1, 0
        target_mbti = target_mbti.upper()
        if target_mbti not in ['E', 'N', 'F', 'P']:
            if target_mbti in ['I', 'S', 'T', 'J']:
                t_value, f_value = 0, 1
            else:
                raise ValueError ("Wrong mbti type. Try different type instead.")

        data['MBTI'] = data['MBTI'].str     \
            .contains(target_mbti)          \
                .replace({True: t_value, False: f_value})

        col_name = None
        if target_mbti in ('E', 'I'):
            col_name = 'I/E'
        elif target_mbti in ('N', 'S'):
            col_name = 'S/N'
        elif target_mbti in ('F', 'T'):
            col_name = 'T/F'
        else:
            col_name = 'J/P'
        data.rename(columns = {'MBTI':col_name}, inplace=True)
        
        return col_name

    def tokenize(self, data: pd.DataFrame):

        def tokenize_per_sentence(series: pd.Series) -> str:
            selected_question = self.question_data.iloc[series['Q_number'] - 1].Question
            selected_answer = series['Answer']
            
            return self.tokenizer(selected_question, selected_answer)
        
        data['QandA'] =  data.apply(tokenize_per_sentence, axis=1)
        data = data.drop(labels='Answer', axis=1)
        return data

In [7]:
data_path = "data/example_train.csv"
question_path = "data/question_filtered.csv"

data = ddDataset(
   data_path,
   question_path,
   target_mbti = "E"
)

data :    Data_ID  User_ID  Gender  Age  MBTI  Q_number                                             Answer
0        1        1       1   30  INFP         1  <아니다> 어릴 때 왕따 당한 경험이 있고 외부 활동을 좋아하지 않기 때문에 소수의...
1        2        1       1   30  INFP         2  <중립>  다양한 관심사를 탐구하진 않지만 대체로 자연과 역사에 관련된 것을 좋아하...
2        3        1       1   30  INFP         3  <그렇다> 감정 이입이 잘되어 코미디 영화에서 사람이 울고 있을 때도 울기 때문에 ...
3        4        1       1   30  INFP         4  <중립> 대비책을 세우긴 하는데 세우다가 마는 편입니다. 일의 변수가 생길 수 있고...
4        5        1       1   30  INFP         5  <아니다> 평정심을 유지 못 하는 편입니다. 머릿속은 백지화가 된 상태로 말도 제대...
--------origin---------
                                                 self                                              other
0   <아니다> 어릴 때 왕따 당한 경험이 있고 외부 활동을 좋아하지 않기 때문에 소수의...  <아니다> 어릴 때 왕따 당한 경험이 있고 외부 활동을 좋아하지 않기 때문에 소수의...
1   <중립> 다양한 관심사를 탐구하진 않지만 대체로 자연과 역사에 관련된 것을 좋아하며...  <중립>  다양한 관심사를 탐구하진 않지만 대체로 자연과 역사에 관련된 것을 좋아하...
2   <그렇다> 감정이입이 잘 되어 코미디 영화에서 사람이 울고 있을 때도 울기 때문에 ... 

주기적으로 새로운 친구를 만드나요? 경험을 비추어봤을 때 어떤지와 그러한 이유가 궁금해요.
<아니다> 어릴 때 왕따 당한 경험이 있고 외부 활동을 좋아하지 않기 때문에 소수의 친구와만 지냅니다.
input_ids : [2, 7267, 11187, 3824, 3949, 2138, 4577, 2075, 2182, 35, 4043, 2069, 9695, 2051, 3072, 2069, 904, 19093, 2522, 3626, 2470, 3782, 2116, 5333, 2097, 2182, 18, 3, 32, 3614, 2062, 34, 8115, 904, 18149, 6391, 4043, 2052, 1513, 2088, 4993, 3746, 2069, 6992, 2118, 1380, 2015, 3624, 2170, 6700, 2079, 3949, 2522, 2154, 1583, 26775, 18, 3]
token_type_ids : [0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1]
attention_mask : [1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1]
['[CLS]', '주기', '##적으로', '새로운', '친구', '##를', '만드', '##나', '##요', '?', '경험', '##을', '비추', '##어', '##봤', '##을', '때', '어떤지', '##와', '그러', '##한', '이유', '##가', '궁금', '#

In [8]:
# 외부에서 import 했을 때도 잘 동작하는지 확인 --> 된다!
from dataloader import MBTIDataset

data = MBTIDataset(
   data_path,
   question_path,
   target_mbti = "E"
)

In [7]:
datum = data['MBTI'].str.contains('I').replace({True: 1, False: 0})

In [11]:
type(datum)

pandas.core.series.Series

In [12]:
data['MBTI'] = datum

In [13]:
data

,Data_ID,User_ID,Gender,Age,MBTI,Q_number,Answer
0,1,1,1,30,1,1,<아니다> 어릴 때 왕따 당한 경험이 있고 외부 활동을 좋아하지 않기 때문에 소수의...
1,2,1,1,30,1,2,<중립> 다양한 관심사를 탐구하진 않지만 대체로 자연과 역사에 관련된 것을 좋아하...
2,3,1,1,30,1,3,<그렇다> 감정 이입이 잘되어 코미디 영화에서 사람이 울고 있을 때도 울기 때문에 ...
3,4,1,1,30,1,4,<중립> 대비책을 세우긴 하는데 세우다가 마는 편입니다. 일의 변수가 생길 수 있고...
4,5,1,1,30,1,5,<아니다> 평정심을 유지 못 하는 편입니다. 머릿속은 백지화가 된 상태로 말도 제대...
...,...,...,...,...,...,...,...
91,104,2,1,40,0,44,<아니다> 계획에 따라 다르겠지만 다시 계획을 실천하기보다는 다른 계획을 이루기 위...
92,105,2,1,40,0,45,"<그렇다> 저는 오래전 후회되는 일을 생각합니다. 어릴 적 더 저에게, 부모님께 열..."
93,106,2,1,40,0,46,<그렇다> 저는 삶의 이유를 깊이 생각하지 않아요. 인간의 존재와 삶은 자연스레 흘...
94,107,2,1,40,0,47,<그렇다> 저는 감정과의 싸움에서 질 때가 많아요. 스스로 조절이 안되어 화낼 때가...


In [7]:
print(data['Age'].mean())
print(data['Age'].std())
data['Age'] = (data['Age'] - data['Age'].mean()) / data['Age'].std()
print(data['Age'])

35.0
5.026246899500346
0    -0.994778
1    -0.994778
2    -0.994778
3    -0.994778
4    -0.994778
        ...   
91    0.994778
92    0.994778
93    0.994778
94    0.994778
95    0.994778
Name: Age, Length: 96, dtype: float64


In [10]:
selected_cols = ['Gender', 'Age', 'Q_number', 'Answer']
data = data[selected_cols]
data

,Gender,Age,Q_number,Answer
0,1,-0.994778,1,<아니다> 어릴 때 왕따 당한 경험이 있고 외부 활동을 좋아하지 않기 때문에 소수의...
1,1,-0.994778,2,<중립> 다양한 관심사를 탐구하진 않지만 대체로 자연과 역사에 관련된 것을 좋아하...
2,1,-0.994778,3,<그렇다> 감정 이입이 잘되어 코미디 영화에서 사람이 울고 있을 때도 울기 때문에 ...
3,1,-0.994778,4,<중립> 대비책을 세우긴 하는데 세우다가 마는 편입니다. 일의 변수가 생길 수 있고...
4,1,-0.994778,5,<아니다> 평정심을 유지 못 하는 편입니다. 머릿속은 백지화가 된 상태로 말도 제대...
...,...,...,...,...
91,1,0.994778,44,<아니다> 계획에 따라 다르겠지만 다시 계획을 실천하기보다는 다른 계획을 이루기 위...
92,1,0.994778,45,"<그렇다> 저는 오래전 후회되는 일을 생각합니다. 어릴 적 더 저에게, 부모님께 열..."
93,1,0.994778,46,<그렇다> 저는 삶의 이유를 깊이 생각하지 않아요. 인간의 존재와 삶은 자연스레 흘...
94,1,0.994778,47,<그렇다> 저는 감정과의 싸움에서 질 때가 많아요. 스스로 조절이 안되어 화낼 때가...


In [15]:
a_1 = data.loc[data['Answer'].str.contains('<그렇다>')]
a_2 = data.loc[data['Answer'].str.contains('<중립>')]
a_3 = data.loc[data['Answer'].str.contains('<아니다>')]
data_filtered = data.drop(a_1.index)
data_filtered = data_filtered.drop(a_2.index)
data_filtered = data_filtered.drop(a_3.index)
data_filtered

,Data_ID,User_ID,Gender,Age,MBTI,Q_number,Answer
874,1091,19,1,40,ISTJ,11,<어렵다> 저는 새로운 사람과 대화하기가 어렵습니다. 그래서 관심이 가는 사람이 있...
1249,1562,27,0,40,INTP,2,< 그렇다> 저는 여러 가지 일에 관심을 두고 있어요 이유는 나이가 40대 이직은 ...
1403,1752,30,1,30,ENFP,12,<아니요> 저는 예술 작품의 다양한 해석에 대해 토론하는 일에 관심이 많습니다. 영...
2651,3312,56,1,30,ISTP,12,"<그렇자> 예술 분야는 극히 주관적인 해석이 가능한 분야라고 생각하며, 굳이 남들과..."
3083,3852,65,0,30,ENFP,12,<아니요> 예술 작품을 볼 때 시간을 많이 들여서 보는 편이고 혼자 가는 것도 좋지...
3971,4956,83,1,40,INTJ,36,<아니요> 저는 파티보다는 혼자서 쉬는 시간을 선택합니다. 이유는 파티를 하고 나면...
4186,5231,88,1,40,ISFP,11,<아니요> 관심이 가는 사람이 별로 없으며 먼저 대화하는 일은 거의 없어요. 새 직...
5057,6318,106,1,30,ENTJ,18,<아니오> 나보다 남의 성공을 돕는 일에 더 큰 만족감을 느끼지 않습니다. 그 이유...
5069,6330,106,1,30,ENTJ,30,<아니오> 저는 사후 세계에 대한 질문이 흥미롭지는 않습니다. 그 이유는 지금 당장...
5118,6391,107,1,40,ESTP,31,<중립 > 요즘은 혼자도 괜찮다고 생각된다. 혼자 있으면 오롯이 나에게 집중할 수 ...


In [14]:
a = 'sa'
a.upper()

'SA'

In [17]:
's' in ('t','a')

False

In [18]:
data.rename(columns = {'MBTI':'AAA'}, inplace=True)

In [19]:
data

,Data_ID,User_ID,Gender,Age,AAA,Q_number,Answer
0,1,1,1,30,1,1,<아니다> 어릴 때 왕따 당한 경험이 있고 외부 활동을 좋아하지 않기 때문에 소수의...
1,2,1,1,30,1,2,<중립> 다양한 관심사를 탐구하진 않지만 대체로 자연과 역사에 관련된 것을 좋아하...
2,3,1,1,30,1,3,<그렇다> 감정 이입이 잘되어 코미디 영화에서 사람이 울고 있을 때도 울기 때문에 ...
3,4,1,1,30,1,4,<중립> 대비책을 세우긴 하는데 세우다가 마는 편입니다. 일의 변수가 생길 수 있고...
4,5,1,1,30,1,5,<아니다> 평정심을 유지 못 하는 편입니다. 머릿속은 백지화가 된 상태로 말도 제대...
...,...,...,...,...,...,...,...
91,104,2,1,40,0,44,<아니다> 계획에 따라 다르겠지만 다시 계획을 실천하기보다는 다른 계획을 이루기 위...
92,105,2,1,40,0,45,"<그렇다> 저는 오래전 후회되는 일을 생각합니다. 어릴 적 더 저에게, 부모님께 열..."
93,106,2,1,40,0,46,<그렇다> 저는 삶의 이유를 깊이 생각하지 않아요. 인간의 존재와 삶은 자연스레 흘...
94,107,2,1,40,0,47,<그렇다> 저는 감정과의 싸움에서 질 때가 많아요. 스스로 조절이 안되어 화낼 때가...


In [20]:
def add_one(a):
  return a+1

In [21]:
data['Age'] = data['Age'].apply(add_one)

In [22]:
data

,Data_ID,User_ID,Gender,Age,AAA,Q_number,Answer
0,1,1,1,31,1,1,<아니다> 어릴 때 왕따 당한 경험이 있고 외부 활동을 좋아하지 않기 때문에 소수의...
1,2,1,1,31,1,2,<중립> 다양한 관심사를 탐구하진 않지만 대체로 자연과 역사에 관련된 것을 좋아하...
2,3,1,1,31,1,3,<그렇다> 감정 이입이 잘되어 코미디 영화에서 사람이 울고 있을 때도 울기 때문에 ...
3,4,1,1,31,1,4,<중립> 대비책을 세우긴 하는데 세우다가 마는 편입니다. 일의 변수가 생길 수 있고...
4,5,1,1,31,1,5,<아니다> 평정심을 유지 못 하는 편입니다. 머릿속은 백지화가 된 상태로 말도 제대...
...,...,...,...,...,...,...,...
91,104,2,1,41,0,44,<아니다> 계획에 따라 다르겠지만 다시 계획을 실천하기보다는 다른 계획을 이루기 위...
92,105,2,1,41,0,45,"<그렇다> 저는 오래전 후회되는 일을 생각합니다. 어릴 적 더 저에게, 부모님께 열..."
93,106,2,1,41,0,46,<그렇다> 저는 삶의 이유를 깊이 생각하지 않아요. 인간의 존재와 삶은 자연스레 흘...
94,107,2,1,41,0,47,<그렇다> 저는 감정과의 싸움에서 질 때가 많아요. 스스로 조절이 안되어 화낼 때가...


In [23]:
q = pd.read_csv('Question.csv')
q

,index,index.1,Question
0,1,1,주기적으로 새로운 친구를 만드나요? 경험을 비추어봤을 때 어떤지와 그러한 이유가 궁...
1,2,2,자유 시간 중 상당 부분을 다양한 관심사를 탐구하는 데 할애하나요? 요즘 어떤 관심...
2,3,3,다른 사람이 울고 있는 모습을 보면 자신도 울고 싶어질 때가 많나요? 이런 상황에서...
3,4,4,일이 잘못될 때를 대비해 여러 대비책을 세우는 편인가요? 이유는 무엇인가요.
4,5,5,압박감이 심한 환경에서도 평정심을 유지하는 편인가요? 최근 경험을 말씀해주세요.
5,6,6,파티나 행사에서 새로운 사람에게 먼저 자신을 소개하기보다는 주로 이미 알고 있는 사...
6,7,7,하나의 프로젝트를 완전히 완료한 후 다른 프로젝트를 시작하는 편인가요? 경험에 비추...
7,8,8,매우 감상적인 편인가요? 그렇게 생각한 이유는 무엇인가요.
8,9,9,일정이나 목록으로 계획을 세우는 일을 좋아하나요? 예를 들어 다이어리나 전자적 도구...
9,10,10,작은 실수로도 자신의 능력이나 지식을 의심하곤 하나요? 최근에 그러한 경험이 있다면...


In [32]:
q.drop(labels=['index', 'index.1'], axis = 1)

,Question
0,주기적으로 새로운 친구를 만드나요? 경험을 비추어봤을 때 어떤지와 그러한 이유가 궁...
1,자유 시간 중 상당 부분을 다양한 관심사를 탐구하는 데 할애하나요? 요즘 어떤 관심...
2,다른 사람이 울고 있는 모습을 보면 자신도 울고 싶어질 때가 많나요? 이런 상황에서...
3,일이 잘못될 때를 대비해 여러 대비책을 세우는 편인가요? 이유는 무엇인가요.
4,압박감이 심한 환경에서도 평정심을 유지하는 편인가요? 최근 경험을 말씀해주세요.
5,파티나 행사에서 새로운 사람에게 먼저 자신을 소개하기보다는 주로 이미 알고 있는 사...
6,하나의 프로젝트를 완전히 완료한 후 다른 프로젝트를 시작하는 편인가요? 경험에 비추...
7,매우 감상적인 편인가요? 그렇게 생각한 이유는 무엇인가요.
8,일정이나 목록으로 계획을 세우는 일을 좋아하나요? 예를 들어 다이어리나 전자적 도구...
9,작은 실수로도 자신의 능력이나 지식을 의심하곤 하나요? 최근에 그러한 경험이 있다면...


In [40]:
q.set_index('index').drop('index.1', axis=1).to_csv('question2.csv', index=False)

In [43]:
qq = pd.read_csv('question.csv')

In [44]:
qq

,index,Question
0,1,주기적으로 새로운 친구를 만드나요? 경험을 비추어봤을 때 어떤지와 그러한 이유가 궁...
1,2,자유 시간 중 상당 부분을 다양한 관심사를 탐구하는 데 할애하나요? 요즘 어떤 관심...
2,3,다른 사람이 울고 있는 모습을 보면 자신도 울고 싶어질 때가 많나요? 이런 상황에서...
3,4,일이 잘못될 때를 대비해 여러 대비책을 세우는 편인가요? 이유는 무엇인가요.
4,5,압박감이 심한 환경에서도 평정심을 유지하는 편인가요? 최근 경험을 말씀해주세요.
5,6,파티나 행사에서 새로운 사람에게 먼저 자신을 소개하기보다는 주로 이미 알고 있는 사...
6,7,하나의 프로젝트를 완전히 완료한 후 다른 프로젝트를 시작하는 편인가요? 경험에 비추...
7,8,매우 감상적인 편인가요? 그렇게 생각한 이유는 무엇인가요.
8,9,일정이나 목록으로 계획을 세우는 일을 좋아하나요? 예를 들어 다이어리나 전자적 도구...
9,10,작은 실수로도 자신의 능력이나 지식을 의심하곤 하나요? 최근에 그러한 경험이 있다면...


In [51]:
qq.iloc[0].Question

'주기적으로 새로운 친구를 만드나요? 경험을 비추어봤을 때 어떤지와 그러한 이유가 궁금해요.'